In [181]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval

# Análise Exploratória de Dados Imobiliários

## 01. Conjunto de Dados : Contexto


Escrever uma breve descrição sobre os dados e algumas hipóteses

In [182]:
pd.set_option('display.max_columns', 30)

In [183]:
df = pd.read_csv('dados_imoveis_sp.csv')
df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos
0,"['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...",['101'],2574084550,[1],"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",[],[2],['111'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
1,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",['140'],2583748663,[2],"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",[2],[4],[],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
2,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",['50'],2562971980,[1],"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",[0],[1],['50'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
3,"['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...",['58'],2580478200,[1],"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",[],[1],[],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
4,"['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...",['64'],2583729583,[1],"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",[],[1],['80'],[2],"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."


## 02. Limpeza e Tratamento de dados

Os tipos de dados em todas as colunas deste dataset estão no formato **string**. Os dados numéricos precisam de uma **conversão de tipo**.

In [184]:
cols = ['usableAreas','parkingSpaces','suites','bathrooms','totalAreas','bedrooms']

for var in cols:
    s_extracted_digits = df[var].str.extract('\[[\']{0,1}(\d*)[\']{0,1}\]').squeeze()
    df[var] = s_extracted_digits.apply(lambda x: int(x) if x.isdigit() else np.nan)
    
df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos
0,"['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...",101,2574084550,1.0,"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",NaN,2,111.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
1,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",140,2583748663,2.0,"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",2.0,4,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
2,"['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...",50,2562971980,1.0,"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",0.0,1,50.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
3,"['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...",58,2580478200,1.0,"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",NaN,1,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."
4,"['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...",64,2583729583,1.0,"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",NaN,1,80.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant..."


Diversas colunas também possuem bastante informações que nos ajudaram a melhor caracterizar cada imóvel.
No entanto, essas informações estão em formato bruto, "raw data", e precisam de um tratamento.

A coluna *'address'* possui características que lembram um dicionário. Mas seu tipo é string. 
Precisamos tratar essas linhas para navegar e extrair seus atributos com facilidade.

In [185]:
print(df['address'][0],'\n')
print('Tipo de dado:',type(df['address'][0]))

{'country': 'BR', 'zipCode': '04734003', 'geoJson': '', 'city': 'São Paulo', 'streetNumber': '1850', 'level': 'STREET', 'precision': 'ROOFTOP', 'confidence': 'VALID_STREET', 'stateAcronym': 'SP', 'source': 'CORREIOS', 'point': {'lon': -46.695829, 'source': 'GOOGLE', 'lat': -23.638282}, 'ibgeCityId': '', 'zone': 'Zona Sul', 'street': 'Avenida Adolfo Pinheiro', 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Santo Amaro', 'district': '', 'name': '', 'state': 'São Paulo', 'neighborhood': 'Santo Amaro', 'poisList': ['BS:Graham Bell C/B', 'BS:Graham Bell B/C', 'BS:Rua Verbo Divino, 61', 'BS:Américo Brasiliense C/B', 'BS:Parada Marechal Deodoro 2 - B/C', 'TS:Graham Bell C/B', 'TS:Graham Bell B/C', 'TS:Rua Verbo Divino, 61', 'TS:Américo Brasiliense C/B', 'TS:Parada Marechal Deodoro 2 - B/C', 'CS:7 Molinos', 'CS:Casa de Pães Neblina Paulista', 'CS:Casa de Bolo', 'CS:Berna', 'CS:Gêmel', 'VP:Kennel Club'], 'pois': [], 'valuableZones': [{'city': 'São Paulo', 'zone': 'Zona Sul', 'name': 'Cháca

O comando **literal_eval** é um interessante comando da biblioteca **ast – Abstract Syntax Trees**. 

Ele avalia uma string contendo uma expressão Python e a executa.

In [186]:
print(literal_eval(df['address'][0]),'\n')
print('Tipo de dado:',type(literal_eval(df['address'][0])))

{'country': 'BR', 'zipCode': '04734003', 'geoJson': '', 'city': 'São Paulo', 'streetNumber': '1850', 'level': 'STREET', 'precision': 'ROOFTOP', 'confidence': 'VALID_STREET', 'stateAcronym': 'SP', 'source': 'CORREIOS', 'point': {'lon': -46.695829, 'source': 'GOOGLE', 'lat': -23.638282}, 'ibgeCityId': '', 'zone': 'Zona Sul', 'street': 'Avenida Adolfo Pinheiro', 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Santo Amaro', 'district': '', 'name': '', 'state': 'São Paulo', 'neighborhood': 'Santo Amaro', 'poisList': ['BS:Graham Bell C/B', 'BS:Graham Bell B/C', 'BS:Rua Verbo Divino, 61', 'BS:Américo Brasiliense C/B', 'BS:Parada Marechal Deodoro 2 - B/C', 'TS:Graham Bell C/B', 'TS:Graham Bell B/C', 'TS:Rua Verbo Divino, 61', 'TS:Américo Brasiliense C/B', 'TS:Parada Marechal Deodoro 2 - B/C', 'CS:7 Molinos', 'CS:Casa de Pães Neblina Paulista', 'CS:Casa de Bolo', 'CS:Berna', 'CS:Gêmel', 'VP:Kennel Club'], 'pois': [], 'valuableZones': [{'city': 'São Paulo', 'zone': 'Zona Sul', 'name': 'Cháca

A coluna *'pricingInfos'* pode possuir dois dicionários. Um contendo preço de aluguel, e outro com preço de compra.
Estamos interessados em pegar apenas o preço de aluguel.

In [187]:
literal_eval(df['pricingInfos'][91])

[{'rentalInfo': {'period': 'MONTHLY',
   'warranties': ['INSURANCE_GUARANTEE', 'GUARANTOR']},
  'yearlyIptu': '120',
  'price': '410000',
  'businessType': 'SALE',
  'monthlyCondoFee': '700'},
 {'rentalInfo': {'period': 'MONTHLY',
   'warranties': ['INSURANCE_GUARANTEE', 'GUARANTOR'],
   'monthlyRentalTotalPrice': '2400'},
  'yearlyIptu': '120',
  'price': '1700',
  'businessType': 'RENTAL',
  'monthlyCondoFee': '700'}]

A coluna *'amenities'* apresenta as facilidades que cada imóvel pode oferecer.
- Como podemos contar a frequência para nosso conjunto dados?

In [188]:
df['amenities'][:5]

0    ['PETS_ALLOWED', 'ELEVATOR', 'GARDEN', 'ELECTR...
1    ['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...
2    ['POOL', 'FURNISHED', 'BARBECUE_GRILL', 'ELEVA...
3    ['POOL', 'BARBECUE_GRILL', 'GATED_COMMUNITY', ...
4    ['PETS_ALLOWED', 'GATED_COMMUNITY', 'ELECTRONI...
Name: amenities, dtype: object

In [189]:
def extract_neighborhood(address):
    address = literal_eval(address)
    neighborhood = address['neighborhood']
    
    return neighborhood

In [190]:
def extract_zone(address):
    address = literal_eval(address)
    zone = address['zone']
    
    return zone

In [191]:
def extract_zipcode(address):
    address = literal_eval(address)
    zipCode = address['zipCode']
    
    return zipCode

In [192]:
def get_rental_price(pricingInfos):
    price = [info['price'] for info\
     in literal_eval(pricingInfos)\
     if info['businessType'] == 'RENTAL'][0]
    
    return float(price)

In [193]:
def strings_para_lista(string):
    lista = string.replace(' ','').replace("'","")
    lista = lista.replace('[','').replace(']','').split(',')
    
    return lista

In [194]:
df['amenities'] = df['amenities'].apply(strings_para_lista)

In [195]:
print('Top 10 Facilidades dos Imóveis de São Paulo:')
print(df['amenities'].explode().value_counts()[:10])

Top 10 Facilidades dos Imóveis de São Paulo:
ELEVATOR           5129
POOL               4613
PARTY_HALL         4110
BARBECUE_GRILL     3961
SERVICE_AREA       3729
GYM                3687
PLAYGROUND         3357
GARDEN             3203
INTERCOM           2899
GATED_COMMUNITY    2854
Name: amenities, dtype: int64


In [196]:
top10_amenities = list(df['amenities'].explode().value_counts()[:10].index)

In [197]:
def has_amenity(amenities,amenity):
    if amenity in amenities:
        return 1
    else:
        return 0

In [198]:
# Cria uma coluna para cada uma das 10 principais facilidades
# com valor binário, representando ausência ou ocorrência da mesma
for amenity in top10_amenities:
    df[amenity.lower()] = df['amenities'].apply(has_amenity,amenity=amenity)

In [199]:
df['zipCode'] = df['address'].apply(extract_zipcode)
df['zone'] = df['address'].apply(extract_zone)
df['neighborhood'] = df['address'].apply(extract_neighborhood)

In [200]:
df.head()

,amenities,usableAreas,id,parkingSpaces,address,suites,bathrooms,totalAreas,bedrooms,pricingInfos,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood
0,"[PETS_ALLOWED, ELEVATOR, GARDEN, ELECTRONIC_GA...",101,2574084550,1.0,"{'country': 'BR', 'zipCode': '04734003', 'geoJ...",NaN,2,111.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,0,0,0,0,0,0,1,0,0,04734003,Zona Sul,Santo Amaro
1,"[POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GY...",140,2583748663,2.0,"{'country': 'BR', 'zipCode': '01307000', 'geoJ...",2.0,4,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,1,1,1,0,1,1,1,0,0,01307000,Centro,Consolação
2,"[POOL, FURNISHED, BARBECUE_GRILL, ELEVATOR, GA...",50,2562971980,1.0,"{'country': 'BR', 'zipCode': '01209010', 'geoJ...",0.0,1,50.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,1,1,1,0,1,1,1,0,1,01209010,Centro,Santa Efigênia
3,"[POOL, BARBECUE_GRILL, GATED_COMMUNITY, GYM, G...",58,2580478200,1.0,"{'country': 'BR', 'zipCode': '01127000', 'geoJ...",NaN,1,NaN,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",0,1,1,1,0,1,1,1,0,1,01127000,Centro,Bom Retiro
4,"[PETS_ALLOWED, GATED_COMMUNITY, ELECTRONIC_GAT...",64,2583729583,1.0,"{'country': 'BR', 'zipCode': '05435001', 'geoJ...",NaN,1,80.0,2,"[{'rentalInfo': {'period': 'MONTHLY', 'warrant...",1,0,0,0,0,0,0,0,0,1,05435001,Zona Oeste,Sumarezinho


In [201]:
df['rental_price'] = df['pricingInfos'].apply(get_rental_price)

In [202]:
df.drop(['amenities','address','pricingInfos'], axis=1,inplace=True)

In [203]:
df.head()

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood,rental_price
0,101,2574084550,1.0,NaN,2,111.0,2,1,0,0,0,0,0,0,1,0,0,04734003,Zona Sul,Santo Amaro,2300.0
1,140,2583748663,2.0,2.0,4,NaN,2,1,1,1,1,0,1,1,1,0,0,01307000,Centro,Consolação,9500.0
2,50,2562971980,1.0,0.0,1,50.0,2,1,1,1,1,0,1,1,1,0,1,01209010,Centro,Santa Efigênia,3000.0
3,58,2580478200,1.0,NaN,1,NaN,2,0,1,1,1,0,1,1,1,0,1,01127000,Centro,Bom Retiro,1900.0
4,64,2583729583,1.0,NaN,1,80.0,2,1,0,0,0,0,0,0,0,0,1,05435001,Zona Oeste,Sumarezinho,2400.0


Existem 'id' repetidos. Isso pode indicar repetição de padrões na base. De fato, existem no máximo 72 repetições na base

In [204]:
df['id'].unique().shape

(9928,)

Mas a função duplicated mostra que não existe 71 linhas com informações duplicadas, mas analisando as duplicatas na coluna 'id' vemos que tem 72 repetições. Analisando mais a fundo podemos ver que existe um mesmo imóvel como 'id'= 2583627481 que tem diferentes valore de aluguel e por isso a função dupllicated para o dataframe e para a coluna 'id' tiveram tamanhos distintos. Escolheu-se retirar todos os 'id' repetidos.

In [205]:
df_duplicated_rows = df[df.duplicated()]
df_duplicated_rows.shape

(71, 21)

In [206]:
df_duplicated_id = df[df['id'].duplicated()]
df_duplicated_id.shape

(72, 21)

In [207]:
get_idx = 0
for i in df_duplicated_id.index:
    if i not in df_duplicated_rows.index:
        get_idx = i
df[df['id']==df_duplicated_id.loc[get_idx,:]['id']]

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood,rental_price
6707,92,2583627481,2.0,1.0,3,92.0,3,1,1,0,1,1,0,1,1,0,1,01153000,Centro,Barra Funda,3000.0
7099,92,2583627481,2.0,1.0,3,92.0,3,1,1,0,1,1,0,1,1,0,1,01153000,Centro,Barra Funda,2900.0


In [208]:
df = df.drop(df_duplicated_id.index)
df.head()

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,zipCode,zone,neighborhood,rental_price
0,101,2574084550,1.0,NaN,2,111.0,2,1,0,0,0,0,0,0,1,0,0,04734003,Zona Sul,Santo Amaro,2300.0
1,140,2583748663,2.0,2.0,4,NaN,2,1,1,1,1,0,1,1,1,0,0,01307000,Centro,Consolação,9500.0
2,50,2562971980,1.0,0.0,1,50.0,2,1,1,1,1,0,1,1,1,0,1,01209010,Centro,Santa Efigênia,3000.0
3,58,2580478200,1.0,NaN,1,NaN,2,0,1,1,1,0,1,1,1,0,1,01127000,Centro,Bom Retiro,1900.0
4,64,2583729583,1.0,NaN,1,80.0,2,1,0,0,0,0,0,0,0,0,1,05435001,Zona Oeste,Sumarezinho,2400.0


# Tratamento de Outliers

In [209]:
df.describe()

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,rental_price
count,9928.000000,9.928000e+03,9534.000000,8427.000000,9928.000000,7999.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000
mean,83.856366,2.576503e+09,1.322320,0.856889,1.930399,99.252407,2.083199,0.513699,0.461523,0.411765,0.396555,0.372683,0.369359,0.335919,0.320608,0.290290,0.285556,4484.447623
std,71.716484,5.308270e+07,1.114758,0.982001,1.190308,170.231681,0.839207,0.499837,0.498542,0.492178,0.489207,0.483543,0.482656,0.472334,0.466734,0.453919,0.451702,10535.670046
min,10.000000,6.506009e+07,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,500.000000
25%,49.000000,2.574103e+09,1.000000,0.000000,1.000000,50.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1800.000000
50%,65.000000,2.582125e+09,1.000000,1.000000,2.000000,70.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2760.000000
75%,96.000000,2.587236e+09,2.000000,1.000000,2.000000,103.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4500.000000
max,3300.000000,2.591836e+09,45.000000,20.000000,10.000000,6000.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,710000.000000


Identificando outliers e substituindo por um NaN

In [211]:
cols = ['usableAreas','parkingSpaces','suites','bathrooms','totalAreas','bedrooms', 'rental_price']
for col,rpl_by in zip(cols,replace_by):
    median = df[col].median()
    Q1 = df[col].quantile(q=0.25)
    Q3 = df[col].quantile(q=0.75)
    IQ = Q3-Q1
    lim_sup = Q3+1.5*IQ
    lim_inf = Q1-1.5*IQ

    df[col]=np.where(((df[col]<lim_inf)|(df[col]>lim_sup)),np.nan,df[col])

In [212]:
df.describe()

,usableAreas,id,parkingSpaces,suites,bathrooms,totalAreas,bedrooms,elevator,pool,party_hall,barbecue_grill,service_area,gym,playground,garden,intercom,gated_community,rental_price
count,9154.000000,9.928000e+03,9202.000000,7658.000000,8816.000000,7343.000000,9927.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9928.000000,9008.000000
mean,69.885842,2.576503e+09,1.200174,0.618699,1.590404,74.268283,2.082704,0.513699,0.461523,0.411765,0.396555,0.372683,0.369359,0.335919,0.320608,0.290290,0.285556,3009.685613
std,31.357299,5.308270e+07,0.757478,0.619710,0.701505,35.442299,0.837797,0.499837,0.498542,0.492178,0.489207,0.483543,0.482656,0.472334,0.466734,0.453919,0.451702,1677.390350
min,10.000000,6.506009e+07,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,500.000000
25%,47.000000,2.574103e+09,1.000000,0.000000,1.000000,49.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1750.000000
50%,63.000000,2.582125e+09,1.000000,1.000000,1.000000,65.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2500.000000
75%,85.000000,2.587236e+09,2.000000,1.000000,2.000000,91.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3800.000000
max,166.000000,2.591836e+09,3.000000,2.000000,3.000000,182.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8540.000000


# Tratamento de valores faltantes

Pelas informações do dataframe, existem 3 colunas com dados faltantes que precisam ser tratados.

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9928 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   usableAreas      9154 non-null   float64
 1   id               9928 non-null   int64  
 2   parkingSpaces    9202 non-null   float64
 3   suites           7658 non-null   float64
 4   bathrooms        8816 non-null   float64
 5   totalAreas       7343 non-null   float64
 6   bedrooms         9927 non-null   float64
 7   elevator         9928 non-null   int64  
 8   pool             9928 non-null   int64  
 9   party_hall       9928 non-null   int64  
 10  barbecue_grill   9928 non-null   int64  
 11  service_area     9928 non-null   int64  
 12  gym              9928 non-null   int64  
 13  playground       9928 non-null   int64  
 14  garden           9928 non-null   int64  
 15  intercom         9928 non-null   int64  
 16  gated_community  9928 non-null   int64  
 17  zipCode       

Separando as colunas que contem dados faltantes em um dataframe

Obtendo as porcentagens de dados faltantes com relação a quantidade total em cada coluna

## Análise dos dados

### Análise univariada

### Análise bivariada

### Análise de correlação